In [1]:
from analysis import MatchAnalysis
import util_strings as utils
from ranking import Ranking


In [2]:
ranking = Ranking('SerieA', '2022-2023')
ranking.read_matches(seasons=4, path=utils.ranking)

ma = MatchAnalysis()
ma.set_ranking(ranking=ranking)

In [3]:
#leggo tutti i match (doppi perché ci sono le statistiche x ogni squadra)
ma.read_matches(utils.merged_statistics) 
#creo una lista di oggetti (lista di squadre)
ma.create_team_dataset()
#divido le partite di casa da quelle in trasferta
ma.divide_and_merge_home_away()
#vado a combinarle
ma.reduce_dataset_with_avg(number = 5, path=utils.dataset_without_text)

Per vedere se funziona il ranking prendo tutte le partite tra due squadre